# BUILD LLM Bootcamp Day 1 

Prerequisite: Run the following commands in a terminal window to create a conda environment.

<pre>
- conda create --name llm-bootcamp -c https://repo.anaconda.com/pkgs/snowflake python=3.9
- conda activate llm-bootcamp
- conda install -c https://repo.anaconda.com/pkgs/snowflake snowflake-snowpark-python pandas notebook
<pre/>


### Install Libraries

In [1]:
!pip install snowflake-ml-python==1.0.12


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 18.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 20.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 20.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 24.0 MB/s eta 0:00:0

In [2]:
!pip install transformers==4.34.0 tokenizers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.6 MB/s eta 0:00:00


### Import Libraries

In [3]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Establish Secure Connection

*NOTE: Update [connection.json](connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [4]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


User                        : USER0046
Role                        : "ROLE_USER0046"
Database                    : "DB_USER0046"
Schema                      : "SCHEMA_LLM"
Warehouse                   : "WH_XS_USER0046"
Snowflake version           : 7.43.0
Snowpark for Python version : 1.9.0


### Reference Llama 2 from Huggingface

In [5]:
options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)
llama_model = llm.LLM(
    model_id_or_path="meta-llama/Llama-2-7b-chat-hf",
    options=options
)


/Users/dtran/anaconda3/envs/llm-bootcamp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Register, Log and Deploy Llama 2 into Snowpark Container Services 

*NOTE: Logging and deploying the same model are one time operations. Once the model is logged and deployed, use ModeReference to get the reference to the model.*

In [6]:
MODEL_NAME    = "LLAMA2_7b_CHAT"
MODEL_VERSION = "NewBaseV2.0"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session,
    database_name=MODEL_REGISTRY_DB,
    schema_name=MODEL_REGISTRY_SCHEMA,
    create_if_not_exists=True)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)


create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


*Note: Deploying model for the first time can take ~25-30mins*

In [7]:
%%time
# Optionally enable INFO log level
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

llama_model_ref.deploy(
    deployment_name="llama_predict",
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

llama_model_ref = model_registry.ModelReference(registry=registry,model_name=MODEL_NAME,model_version=MODEL_VERSION)
llama_model_ref


INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DB_USER0046.SCHEMA_LLM]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [CREATE STAGE IF NOT EXISTS DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_DEPLOYMENTS_S...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [S

CPU times: user 7.44 s, sys: 1.49 s, total: 8.93 s
Wall time: 31min 54s


### Load Data from JSON into Snowflake

*NOTE: Reading data in JSON and storing it in a Snowflake table are one time operations. Once the data is loaded, use Snowpark to load the data from the existing table.*

In [8]:
df = pd.read_json("data/frosty_transcripts.json",lines=True)
sf_df = session.write_pandas(df,'frosty_transcripts',auto_create_table=True,quote_identifiers=False,overwrite=True)
sf_df.to_pandas().head()


INFO:snowflake.connector.cursor:query: [CREATE TEMP STAGE /* Python:snowflake.connector.pandas_tools.write_pandas() */ k...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file:///var/fo...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE TEMP FILE FORMAT xfjsxztkwq /* Python:snowflake.connector.pandas_tools.wr...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT COLUMN_NAME, TYPE FROM table(infer_schema(location=>'@kshyepwpnn', file_f...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 6
INFO:snowflake.connector.cursor:query: [CREATE  TABLE IF NOT EXISTS orsjxfaunb (LANG

,LANGUAGE,TRANSCRIPT,NAME,LOCATION,TOY_LIST,ID
0,EN,"caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays!",Rachel,Sydney,"[\n ""Barbie Science Lab Playset""\n]",0
1,EN,"caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays!",Jessica,Los Angeles,"[\n ""Barbie Dreamhouse 2023""\n]",2
2,EN,"caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: The ninja turtles delivery van seems perfect! Let's add that to the list.\nfrosty: Excellent choice! The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van is now on your wish list. Can you tell me about one of your favorite pastimes or hobbies, Ashley?\ncaller: Sure! I love spending time in nature, going for hikes and exploring new trails.\nfrosty: That sounds like an excellent way to unwind and enjoy the world around you. I hope you have a fantastic holiday season, Ashley! Goodbye!\ncaller: Thank you, Frosty! Happy holidays!",Ashley,Auckland,"[\n ""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""\n]",4
3,EN,"caller: Hey Frosty! I need some help

### Simple Prompt Engineering Example

For every transcript, define summarization instruction for the LLM

In [9]:
begin_prompt = \
"""
[INST] Summarize this transcript in less than 200 words:
"""
end_prompt = " [/INST]"

df_inputs = sf_df.with_column('"input"',F.concat_ws(F.lit(" "),F.lit(begin_prompt),F.col('transcript'),F.lit(end_prompt))).select('"input"')
df_inputs.to_pandas().head()


INFO:snowflake.connector.cursor:query: [SELECT  *  FROM frosty_transcripts]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT concat_ws(' ', '\n[INST] Summarize this transcript in less than 200 words...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 69


,input
0,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
1,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
2,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: The ninja turtles delivery van seems perfect! Let's add that to the list.\nfrosty: Excellent choice! The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van is now on your wish list. Can you tell me about one of your favorite pastimes or hobbies, Ashley?\ncaller: Sure! I love spending time in nature, going for hikes and exploring new trails.\nfrosty: That sounds like an excellent way to unwind and enjoy the world around you. I hope you have a fantastic holiday season, Ashley! Goodbye!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
3,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hey Frosty! I need so

### Inference using Simple Prompt

Pass the summariation instruction to the LLM and examine results of 10 records

In [10]:
df_predict_results = llama_model_ref.predict(deployment_name="llama_predict",data=df_inputs)
df_predict_results.select('"input"','"generated_text"').limit(5).to_pandas()


INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DB_USER0046.SCHEMA_LLM]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT "MODEL_NAME", "MODEL_VERSION...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:quer

,input,generated_text
0,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Rachel from Sydney calls to submit her holiday wish list to Frosty. She is considering the Barbie Science Lab Playset for her daughter, who loves science and Barbie. Frosty suggests the Beast Lab: Shark Beast Creator as an alternative, but Rachel decides to stick with the original choice. Frosty thanks Rachel and wishes her a great day."
1,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Jessica, a caller from Los Angeles, is looking for a toy for her child that promotes imaginative play and creativity. Frosty suggests the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Jessica decides on the Barbie Dreamhouse, and Frosty asks about her favorite holiday memories. Jessica shares a memory of seeing the city's holiday lights"
2,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Con

### Complex Prompt Engineering and Inference Example

For every transcript, define more specific instruction for the LLM

*NOTE: In the results, notice that the output is not consistent across all transcripts. The base model failed to follow the instructions in many of the cases as seen below.*

In [11]:
sf_df = session.table('frosty_transcripts')

begin_prompt = \
"""
[INST] Extract location and list of toys in JSON format:
"""
end_prompt = " [/INST]"

df_inputs = sf_df.with_column('"input"',F.concat_ws(F.lit(" "),F.lit(begin_prompt),F.col('transcript'),F.lit(end_prompt))).select('"input"')
df_predict_results = llama_model_ref.predict(deployment_name="llama_predict",data=df_inputs)
df_predict_results.limit(10).to_pandas()


INFO:snowflake.connector.cursor:query: [SELECT  *  FROM frosty_transcripts]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DB_USER0046.SCHEMA_LLM]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DB_USER0046.SCHEMA_LLM._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( 

,input,generated_text
0,"\n[INST] Extract location and list of toys in JSON format: \n caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Sure! Here is the JSON format for the conversation:\n\n{\n""name"": ""Rachel"",\n""location"": ""Sydney"",\n""wishList"": [\n{\n""name"": ""Barbie Science Lab Playset""\n},\n{\n""name"": ""Beast Lab: Shark Beast Creator""\n}\n]\n}\n\nExplanation:\n\n* ""name"": Rachel is the name of the caller"
1,"\n[INST] Extract location and list of toys in JSON format: \n caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Here is the JSON format for the conversation:\n\n{\n""location"": ""Los Angeles"",\n""listOfToys"": [\n""Barbie Dreamhouse 2023"",\n""Calico Critters Pony's Stylish Hair Salon""\n]\n}\n\nExplanation:\n\n* ""location"": This field represents the location where the conversation took place, which is Los Angeles in this case.\n* ""listOfTo"
2,"\n[INST] Extract location and list of toys in JSON format: \n caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: T

# BUILD LLM Bootcamp Day 2 Preparation

Follow the instructions below to prepare your environment for BUILD LLM Bootcamp Day 2. 

*NOTE:* These operations can take about ~45-60mins depending on your wireless connection.

1) Open terminal window and browse to the folder where you have cloned the repository

2) Change folder to *day2*

3) Run command *`docker build --platform linux/amd64 -t llm-bootcamp .`*

4) Once that image is built locally, run the following commands to push the image to Snowflake Registry

    1) Replace ***your-account-name*** with your account name and ***your-db-name*** with the name of your database ***in lowercase*** and then run the following command

        `docker tag llm-bootcamp:latest sfsenorthamerica-build-spcs3.registry.snowflakecomputing.com/db_user0046/schema_llm/image_repo/llm-bootcamp:latest`

    2) Replace ***your-account-name*** with your account name and run the following command to login using your LLM Bootcamp Snowflake account username and password

        `docker login sfsenorthamerica-build-spcs3.registry.snowflakecomputing.com`
        
    3) Replace ***your-account-name*** with your account name and ***your-db-name*** with the name of your database ***in lowercase*** and then run the following command
    
        `docker push sfsenorthamerica-build-spcs3.registry.snowflakecomputing.com/db_user0046/schema_llm/image_repo/llm-bootcamp:latest`

